In [2]:
import pandas as pd
from os import path
import csv
import os
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np
import re

In [3]:
cd '/scratch/users/shrestp/mover/EPIC_EMR/EPIC_EMR/EMR/'

/scratch/users/shrestp/mover/EPIC_EMR/EPIC_EMR/EMR


In [4]:
# load map and meds dataFrames and drop duplicates
filepath = '/scratch/users/shrestp/mover/EPIC_flowsheets_cleaned/final_map_df.csv'
df_map = pd.read_csv(filepath, index_col=0)
df_map = df_map.drop_duplicates()
df_map['RECORDED_TIME'] = pd.to_datetime(df_map['RECORDED_TIME'])

# original 
#filepath = '/scratch/users/shrestp/mover/EPIC_EMR/EPIC_EMR/EMR/pressor_patient_meds.csv'
#df_meds = pd.read_csv(filepath, index_col=0)
#df_meds = df_meds.drop_duplicates()

# with demographic data
filepath = '/scratch/users/shrestp/mover/EPIC_EMR/EPIC_EMR/EMR/pressors_09-01.csv'
df_meds = pd.read_csv(filepath, index_col=0)
df_meds = df_meds.drop_duplicates()

In [5]:
# filter medications with inclusion criteria
pattern = r'infusion|IJ|injection|IV'
df_meds = df_meds[df_meds['MEDICATION_NM'].str.contains(pattern, case=False, na=False)]

pattern = r'(?i)(lidocaine|nebu|.*-epinephrine|vasopressin infusion \(anesthesia\))'
df_meds = df_meds[~df_meds['MEDICATION_NM'].str.contains(pattern)]

# drop any medications NAN in the dosage column, canceled entry, or not pressor
df_meds = df_meds.dropna(subset=['ADMIN_SIG'])
df_meds = df_meds[df_meds['MAR_ACTION_NM'] != 'Canceled Entry']
df_meds = df_meds[df_meds['DOSE_UNIT_NM'].str.contains('/min', na=False)]
df_meds['MED_ACTION_TIME'] = pd.to_datetime(df_meds['MED_ACTION_TIME'])
# LOG_ID we want: 'eb401383e5774847'

/tmp/ipykernel_119309/3925469348.py:6: UserWarning: This pattern is interpreted as a regular expression, and has match groups. To actually get the groups, use str.extract.
  df_meds = df_meds[~df_meds['MEDICATION_NM'].str.contains(pattern)]


In [6]:
# keep only the MRNs that appear in both map and med dataFrames
map_MRNs = set(df_map['MRN'])
viable_mrns = list(set(df_map['MRN']).intersection(set(df_meds['MRN'])))
df_map = df_map[df_map['MRN'].isin(viable_mrns)]
df_meds = df_meds[df_meds['MRN'].isin(viable_mrns)]

In [8]:
#cols = df_meds.columns.difference(['LOG_ID'])
#df_meds = df_meds.drop_duplicates(subset=cols)

#cols = df_map.columns.difference(['LOG_ID'])
#df_map = df_map.drop_duplicates(subset=cols)

df_meds[['LOG_ID', 'MRN']].nunique()
df_map[['LOG_ID', 'MRN']].nunique()

LOG_ID    6767
MRN       2210
dtype: int64

In [9]:
df_map

,LOG_ID,MRN,FLO_NAME,FLO_DISPLAY_NAME,RECORD_TYPE,RECORDED_TIME,MEAS_VALUE,UNITS
Unnamed: 0,,,,,,,,
1582,da51bbbe44b985b2,b8bef0111ecada9c,Vital Signs,MAP (mmHg),POST-OP,2021-07-30 23:00:00,102.0,NaN
1584,a7ff644f99bb2f2d,b8bef0111ecada9c,Vital Signs,MAP (mmHg),POST-OP,2021-07-30 23:00:00,102.0,NaN
1587,b2bf5178b216dff9,b8bef0111ecada9c,Vital Signs,MAP (mmHg),POST-OP,2021-07-30 23:00:00,102.0,NaN
1588,fef3b94d7a5b1638,b8bef0111ecada9c,Vital Signs,MAP (mmHg),POST-OP,2021-07-30 23:00:00,102.0,NaN
12496,cbdf5d802c3dac7a,69a90854cdddc9eb,Vital Signs,MAP (mmHg),POST-OP,2023-02-05 07:00:00,69.0,NaN
...,...,...,...,...,...,...,...,...
55687710,98e5c0ed8d13f76f,60e6ea6cf06b58d7,Vital Signs,MAP (mmHg),POST-OP,2020-09-02 10:20:00,91.0,NaN
55687711,bfea52329a898a2d,60e6ea6cf06b58d7,Vital Signs,MAP (mmHg),POST-OP,2020-09-02 10:20:00,91.0,NaN
55687733,31d3ed1c9eee9fc3,60e6ea6cf06b58d7,Vital Signs,MAP (mmHg),POST-OP,2020-09-02 10:20:00,91.0,NaN


In [10]:
# read in patient demographics datset
filepath = '/scratch/users/shrestp/mover/EPIC_EMR/EPIC_EMR/EMR/patient_information.csv'
df_patient = pd.read_csv(filepath)
df_patient = df_patient.drop_duplicates()

In [11]:
mrns = list(set(df_meds['MRN']))

In [12]:
df_meds['ORDERING_DATE'].dtype

dtype('O')

In [26]:
import warnings
warnings.filterwarnings('ignore')

# Set up threshold and counter
#thresh = 10
#count = 0

# Iterate through MRNs
df_collection = []
for mrn in mrns:
    print(f'\nChecking patient: {mrn}')
    
    # Filter and process patient data
    temp = df_patient[df_patient['MRN'] == mrn][['HOSP_ADMSN_TIME', 'HOSP_DISCH_TIME']].drop_duplicates()
    temp['HOSP_ADMSN_TIME'] = pd.to_datetime(temp['HOSP_ADMSN_TIME'])
    temp['HOSP_DISCH_TIME'] = pd.to_datetime(temp['HOSP_DISCH_TIME'])
    
    print(f'There are {len(temp)} unique procedures')
    
    # Iterate through each admission-discharge period
    count = 0
    for index, row in temp.iterrows():
        count += 1
        print(f'Checking procedure {count}')
        # Filter medications and maps within the admission period
        df_meds_p = df_meds[df_meds['MRN'] == mrn]
        df_map_p = df_map[df_map['MRN'] == mrn]
        pmeds = df_meds[(df_meds['MRN'] == mrn) & 
                        (df_meds['MED_ACTION_TIME'].between(row['HOSP_ADMSN_TIME'], row['HOSP_DISCH_TIME']))]
        pmaps = df_map[(df_map['MRN'] == mrn) & 
                       (df_map['RECORDED_TIME'].between(row['HOSP_ADMSN_TIME'], row['HOSP_DISCH_TIME']))]

        # Check if any data is available
        if pmeds.empty or pmaps.empty:
            print("No valid map or med measurements")
            #break
            continue

        # Merge data based on the smaller dataset to minimize merge operations
        if len(pmeds) > len(pmaps):
            time_to_use = 'MED_ACTION_TIME'
            df_combined = pd.merge_asof(pmeds.sort_values('MED_ACTION_TIME').reset_index(drop=True),
                                        pmaps.sort_values('RECORDED_TIME').reset_index(drop=True),
                                        left_on='MED_ACTION_TIME', right_on='RECORDED_TIME', direction='forward') # changed
                                                                                                                # nearest
        else:
            time_to_use = 'RECORDED_TIME'
            df_combined = pd.merge_asof(pmaps.sort_values('RECORDED_TIME').reset_index(drop=True),
                                        pmeds.sort_values('MED_ACTION_TIME').reset_index(drop=True),
                                        left_on='RECORDED_TIME', right_on='MED_ACTION_TIME', direction='backward') # changed
                                                                                                                    # nearest
        # Clean merged data
        if len(df_combined[df_combined[['MED_ACTION_TIME', 'RECORDED_TIME']].isna().any(axis=1)]) > 0:
            print("Found NAN time values in the combined df_combined")
        
        df_combined_cleaned = df_combined.dropna(subset=['MED_ACTION_TIME', 'RECORDED_TIME'])
        df_combined_cleaned['pressor_diff'] = df_combined_cleaned.groupby('MEDICATION_NM')['ADMIN_SIG'].diff()
        
        # Identify valid timestamps
        valid_timestamps = df_combined_cleaned.groupby('MEDICATION_NM').filter(
            lambda x: (x['MEAS_VALUE'].iloc[0] > 70) and 
                      (x['pressor_diff'].lt(0).any()) and 
                      (not x['pressor_diff'].gt(0).any())
        )[time_to_use].unique()
        
        total_hours = -1
        total_hours_hosp = (row['HOSP_DISCH_TIME'] - row['HOSP_ADMSN_TIME']).total_seconds() / 3600
        if len(valid_timestamps) == 0:
            print("No valid timestamps")
            total_hours = 0
        else:
            # Calculate the time differences and normalize by total hospital hours
            df_valid = df_combined_cleaned[df_combined_cleaned[time_to_use].isin(valid_timestamps)]
            df_valid['time_diff'] = df_valid[time_to_use].diff().dt.total_seconds() / 3600
            total_hours = df_valid['time_diff'].sum()
            total_hours /= total_hours_hosp
        
        # Append the results to the collection
        df_collection.append({
            'MRN': mrn,
            'HOSP_ADMSN_TIME': row['HOSP_ADMSN_TIME'],
            'HOSP_DISCH_TIME': row['HOSP_DISCH_TIME'],
            'HOURS': total_hours,
        })
        print(f'total hours: {total_hours}')
        print(f"total hours in hospital: {total_hours_hosp}")
        

# Create the final DataFrame
df_final_hours = pd.DataFrame(df_collection)
df_final_hours.to_csv('hours_fb_df_new.csv')


Checking patient: 38f2abdfacbed586
There are 1 unique procedures
Checking procedure 1
Found NAN time values in the combined df_combined
No valid timestamps
total hours: 0
total hours in hospital: 319.6166666666667

Checking patient: 2d604e5cc7818233
There are 2 unique procedures
Checking procedure 1
No valid map or med measurements
Checking procedure 2
Found NAN time values in the combined df_combined
No valid timestamps
total hours: 0
total hours in hospital: 353.1333333333333

Checking patient: 3b7fd5bd48dae180
There are 1 unique procedures
Checking procedure 1
No valid timestamps
total hours: 0
total hours in hospital: 277.8666666666667

Checking patient: 0ea0ab0b01b01c4c
There are 1 unique procedures
Checking procedure 1
Found NAN time values in the combined df_combined
No valid timestamps
total hours: 0
total hours in hospital: 623.9166666666666

Checking patient: bba97a9988e7a2aa
There are 1 unique procedures
Checking procedure 1
Found NAN time values in the combined df_combined

No valid timestamps
total hours: 0
total hours in hospital: 476.4166666666667

Checking patient: 99330ca6052ec1a4
There are 1 unique procedures
Checking procedure 1
Found NAN time values in the combined df_combined
No valid timestamps
total hours: 0
total hours in hospital: 365.1

Checking patient: 30cc55a44c536b99
There are 1 unique procedures
Checking procedure 1
Found NAN time values in the combined df_combined
total hours: 0.44438186426358767
total hours in hospital: 295.9166666666667

Checking patient: 0610d3e6d478e8fc
There are 2 unique procedures
Checking procedure 1
No valid map or med measurements
Checking procedure 2
No valid timestamps
total hours: 0
total hours in hospital: 148.55

Checking patient: 4302fbdfaeb959dd
There are 1 unique procedures
Checking procedure 1
Found NAN time values in the combined df_combined
No valid timestamps
total hours: 0
total hours in hospital: 80.0

Checking patient: 457ff0dbd0bd8a18
There are 1 unique procedures
Checking procedure 1
Found NAN

Found NAN time values in the combined df_combined
total hours: 0.6141553335016775
total hours in hospital: 462.01666666666665

Checking patient: c5e08055aff28da3
There are 1 unique procedures
Checking procedure 1
No valid timestamps
total hours: 0
total hours in hospital: 229.93333333333334

Checking patient: e0abcf07b8fca3c9
There are 2 unique procedures
Checking procedure 1
Found NAN time values in the combined df_combined
No valid timestamps
total hours: 0
total hours in hospital: 650.4166666666666
Checking procedure 2
No valid map or med measurements

Checking patient: 1c2f04d87315d401
There are 3 unique procedures
Checking procedure 1
Found NAN time values in the combined df_combined
total hours: 0.0004941362498352879
total hours in hospital: 1011.8666666666667
Checking procedure 2
No valid map or med measurements
Checking procedure 3
No valid map or med measurements

Checking patient: 46af3c16d7caf259
There are 1 unique procedures
Checking procedure 1
Found NAN time values in the

Found NAN time values in the combined df_combined
No valid timestamps
total hours: 0
total hours in hospital: 223.2

Checking patient: 1801d9985d23aa30
There are 3 unique procedures
Checking procedure 1
No valid map or med measurements
Checking procedure 2
Found NAN time values in the combined df_combined
No valid timestamps
total hours: 0
total hours in hospital: 129.53333333333333
Checking procedure 3
No valid map or med measurements

Checking patient: f11163c609d6941f
There are 1 unique procedures
Checking procedure 1
Found NAN time values in the combined df_combined
No valid timestamps
total hours: 0
total hours in hospital: 281.75

Checking patient: f8a182dbadf098be
There are 3 unique procedures
Checking procedure 1
No valid map or med measurements
Checking procedure 2
Found NAN time values in the combined df_combined
No valid timestamps
total hours: 0
total hours in hospital: 181.55
Checking procedure 3
No valid map or med measurements

Checking patient: 0cfcb52f13f1830c
There ar

Found NAN time values in the combined df_combined
total hours: 0.04120474835671539
total hours in hospital: 679.5333333333333
Checking procedure 3
No valid map or med measurements

Checking patient: 8039a39629e89008
There are 1 unique procedures
Checking procedure 1
Found NAN time values in the combined df_combined
No valid timestamps
total hours: 0
total hours in hospital: 1515.4

Checking patient: 05118a9865067eca
There are 2 unique procedures
Checking procedure 1
Found NAN time values in the combined df_combined
No valid timestamps
total hours: 0
total hours in hospital: 246.46666666666667
Checking procedure 2
No valid map or med measurements

Checking patient: 4634ccba43e2b85c
There are 2 unique procedures
Checking procedure 1
No valid map or med measurements
Checking procedure 2
Found NAN time values in the combined df_combined
total hours: 0.8921492636474363
total hours in hospital: 150.51666666666668

Checking patient: f5a8cfa68790d9e8
There are 1 unique procedures
Checking proc

...

No valid timestamps
total hours: 0
total hours in hospital: 92.76666666666667

Checking patient: 9cc3d34201aca699
There are 2 unique procedures
Checking procedure 1
No valid map or med measurements
Checking procedure 2
No valid timestamps
total hours: 0
total hours in hospital: 180.5

Checking patient: a63b6187c0cb97ee
There are 1 unique procedures
Checking procedure 1
Found NAN time values in the combined df_combined
No valid timestamps
total hours: 0
total hours in hospital: 555.4833333333333

Checking patient: da55eaa17fdb3c0a
There are 1 unique procedures
Checking procedure 1
Found NAN time values in the combined df_combined
No valid timestamps
total hours: 0
total hours in hospital: 247.83333333333334

Checking patient: 0cb2ee75385523c5
There are 1 unique procedures
Checking procedure 1
Found NAN time values in the combined df_combined
No valid timestamps
total hours: 0
total hours in hospital: 311.31666666666666

Checking patient: 279cf2738219e107
There are 1 unique procedures
Ch

Found NAN time values in the combined df_combined
No valid timestamps
total hours: 0
total hours in hospital: 1682.5166666666667

Checking patient: f17f22fbd0f69058
There are 4 unique procedures
Checking procedure 1
Found NAN time values in the combined df_combined
No valid timestamps
total hours: 0
total hours in hospital: 315.6666666666667
Checking procedure 2
No valid map or med measurements
Checking procedure 3
No valid map or med measurements
Checking procedure 4
No valid map or med measurements

Checking patient: 02ebe777d685f687
There are 1 unique procedures
Checking procedure 1
Found NAN time values in the combined df_combined
No valid timestamps
total hours: 0
total hours in hospital: 707.8

Checking patient: 377c0ea9967b9616
There are 2 unique procedures
Checking procedure 1
Found NAN time values in the combined df_combined
No valid timestamps
total hours: 0
total hours in hospital: 402.93333333333334
Checking procedure 2
No valid map or med measurements

Checking patient: 15

No valid map or med measurements

Checking patient: 9484296afa904ece
There are 1 unique procedures
Checking procedure 1
No valid timestamps
total hours: 0
total hours in hospital: 231.93333333333334

Checking patient: 14b670cf1b83c18d
There are 1 unique procedures
Checking procedure 1
Found NAN time values in the combined df_combined
No valid timestamps
total hours: 0
total hours in hospital: 41.583333333333336

Checking patient: 642bf776bd94e98c
There are 1 unique procedures
Checking procedure 1
total hours: 0.329388560157791
total hours in hospital: 109.85

Checking patient: 7fbc2dceb54e1b5e
There are 1 unique procedures
Checking procedure 1
Found NAN time values in the combined df_combined
No valid timestamps
total hours: 0
total hours in hospital: 562.8

Checking patient: 538ccc476a826386
There are 2 unique procedures
Checking procedure 1
Found NAN time values in the combined df_combined
No valid timestamps
total hours: 0
total hours in hospital: 462.6666666666667
Checking procedur

Found NAN time values in the combined df_combined
No valid timestamps
total hours: 0
total hours in hospital: 327.85

Checking patient: ae78c99ba8555338
There are 1 unique procedures
Checking procedure 1
No valid timestamps
total hours: 0
total hours in hospital: 431.05

Checking patient: 341945e9ad240525
There are 1 unique procedures
Checking procedure 1
Found NAN time values in the combined df_combined
No valid timestamps
total hours: 0
total hours in hospital: 207.38333333333333

Checking patient: df6d8409bfdaa1b2
There are 2 unique procedures
Checking procedure 1
Found NAN time values in the combined df_combined
No valid timestamps
total hours: 0
total hours in hospital: 509.06666666666666
Checking procedure 2
Found NAN time values in the combined df_combined
No valid timestamps
total hours: 0
total hours in hospital: 278.56666666666666

Checking patient: 2a63f2a388bfa4b2
There are 3 unique procedures
Checking procedure 1
No valid map or med measurements
Checking procedure 2
No val

Found NAN time values in the combined df_combined
No valid timestamps
total hours: 0
total hours in hospital: 1031.7
Checking procedure 2
No valid map or med measurements

Checking patient: 5800610092b18862
There are 2 unique procedures
Checking procedure 1
Found NAN time values in the combined df_combined
total hours: 0.5404561588011042
total hours in hospital: 1521.4
Checking procedure 2
Found NAN time values in the combined df_combined
No valid timestamps
total hours: 0
total hours in hospital: 927.9833333333333

Checking patient: 3e56b462de0132a4
There are 4 unique procedures
Checking procedure 1
No valid map or med measurements
Checking procedure 2
No valid map or med measurements
Checking procedure 3
No valid map or med measurements
Checking procedure 4
Found NAN time values in the combined df_combined
No valid timestamps
total hours: 0
total hours in hospital: 732.5833333333334


In [88]:
df_meds.columns

Index(['ENC_TYPE_C', 'ENC_TYPE_NM', 'LOG_ID', 'MRN', 'ORDERING_DATE',
       'ORDER_CLASS_NM', 'MEDICATION_ID', 'DISPLAY_NAME', 'MEDICATION_NM',
       'START_DATE', 'END_DATE', 'ORDER_STATUS_NM', 'RECORD_TYPE',
       'MAR_ACTION_NM', 'MED_ACTION_TIME', 'ADMIN_SIG', 'DOSE_UNIT_NM',
       'MED_ROUTE_NM'],
      dtype='object')

In [55]:
df_map.columns

Index(['LOG_ID', 'MRN', 'FLO_NAME', 'FLO_DISPLAY_NAME', 'RECORD_TYPE',
       'RECORDED_TIME', 'MEAS_VALUE', 'UNITS'],
      dtype='object')

In [75]:
df_combined.columns

Index(['ENC_TYPE_C', 'ENC_TYPE_NM', 'LOG_ID_x', 'MRN_x', 'ORDERING_DATE',
       'ORDER_CLASS_NM', 'MEDICATION_ID', 'DISPLAY_NAME', 'MEDICATION_NM',
       'START_DATE', 'END_DATE', 'ORDER_STATUS_NM', 'RECORD_TYPE_x',
       'MAR_ACTION_NM', 'MED_ACTION_TIME', 'ADMIN_SIG', 'DOSE_UNIT_NM',
       'MED_ROUTE_NM', 'LOG_ID_y', 'MRN_y', 'FLO_NAME', 'FLO_DISPLAY_NAME',
       'RECORD_TYPE_y', 'RECORDED_TIME', 'MEAS_VALUE', 'UNITS'],
      dtype='object')

In [89]:
df_map_p

,LOG_ID,MRN,FLO_NAME,FLO_DISPLAY_NAME,RECORD_TYPE,RECORDED_TIME,MEAS_VALUE,UNITS
Unnamed: 0,,,,,,,,
2556,33163fa8e730e747,af4eb3a7c4c82877,Vital Signs,MAP (mmHg),POST-OP,2021-10-04 03:00:00,93.0,NaN
2557,48a43a20ff56ce42,af4eb3a7c4c82877,Vital Signs,MAP (mmHg),POST-OP,2021-10-04 03:00:00,93.0,NaN
2565,fe48717c3d9c8a15,af4eb3a7c4c82877,Vital Signs,MAP (mmHg),POST-OP,2021-10-04 03:00:00,93.0,NaN
3282,48a43a20ff56ce42,af4eb3a7c4c82877,Vital Signs,MAP (mmHg),POST-OP,2021-10-04 04:00:00,98.0,NaN
3283,fe48717c3d9c8a15,af4eb3a7c4c82877,Vital Signs,MAP (mmHg),POST-OP,2021-10-04 04:00:00,98.0,NaN
...,...,...,...,...,...,...,...,...
56540330,a7b309ece2a59015,af4eb3a7c4c82877,CCP Vital Signs,MAP (mmHg),POST-OP,2022-02-19 11:50:00,88.0,NaN
56540331,48a43a20ff56ce42,af4eb3a7c4c82877,CCP Vital Signs,MAP (mmHg),POST-OP,2022-02-19 11:50:00,88.0,NaN
56540332,fe48717c3d9c8a15,af4eb3a7c4c82877,CCP Vital Signs,MAP (mmHg),POST-OP,2022-02-19 11:50:00,88.0,NaN


In [90]:
df_map_p['RECORDED_TIME'].value_counts()

RECORDED_TIME
2021-12-24 04:07:00    5
2022-02-23 15:37:00    5
2021-12-25 11:00:00    5
2021-12-25 09:16:00    5
2021-12-07 13:13:00    5
                      ..
2021-01-19 04:25:00    1
2021-01-19 08:25:00    1
2021-01-19 15:53:00    1
2021-01-21 15:27:00    1
2021-01-18 07:54:00    1
Name: count, Length: 1365, dtype: int64

In [91]:
df_combined['RECORDED_TIME'].value_counts()

RECORDED_TIME
2021-12-09 03:16:00    5
2022-01-06 11:01:00    5
2022-01-07 12:11:00    5
2022-01-07 09:05:00    5
2022-01-07 07:41:00    5
                      ..
2021-10-03 09:00:00    2
2021-10-29 09:00:00    1
2021-12-26 03:55:00    1
2021-10-09 22:00:00    1
2021-10-07 22:00:00    1
Name: count, Length: 1298, dtype: int64

In [93]:
df_combined[['LOG_ID_x', 'MRN_x', 'LOG_ID_y', 'MRN_y', 'ORDER_STATUS_NM','MAR_ACTION_NM', 'MED_ACTION_TIME',
                        'ADMIN_SIG','RECORDED_TIME', 'MEAS_VALUE' ]]

,LOG_ID_x,MRN_x,LOG_ID_y,MRN_y,ORDER_STATUS_NM,MAR_ACTION_NM,MED_ACTION_TIME,ADMIN_SIG,RECORDED_TIME,MEAS_VALUE
0,48a43a20ff56ce42,af4eb3a7c4c82877,33163fa8e730e747,af4eb3a7c4c82877,Discontinued,Stopped,2021-09-22 04:00:00,0.0,2021-09-29 00:08:00,76.0
1,33163fa8e730e747,af4eb3a7c4c82877,33163fa8e730e747,af4eb3a7c4c82877,Discontinued,Stopped,2021-09-22 04:00:00,0.0,2021-09-29 00:08:00,76.0
2,48a43a20ff56ce42,af4eb3a7c4c82877,33163fa8e730e747,af4eb3a7c4c82877,Discontinued,Stopped,2021-09-22 04:00:00,0.0,2021-09-29 04:55:00,88.0
3,33163fa8e730e747,af4eb3a7c4c82877,33163fa8e730e747,af4eb3a7c4c82877,Discontinued,Stopped,2021-09-22 04:00:00,0.0,2021-09-29 04:55:00,88.0
4,48a43a20ff56ce42,af4eb3a7c4c82877,33163fa8e730e747,af4eb3a7c4c82877,Discontinued,Stopped,2021-09-22 04:00:00,0.0,2021-09-29 08:14:00,72.0
...,...,...,...,...,...,...,...,...,...,...
5883,60f7af654740a548,af4eb3a7c4c82877,60f7af654740a548,af4eb3a7c4c82877,Discontinued,Rate Change,2021-10-26 14:50:00,2.0,2022-04-24 23:06:00,85.0
5884,fe48717c3d9c8a15,af4eb3a7c4c82877,60f7af654740a548,af4eb3a7c4c82877,Discontinued,Rate Change,2021-10-26 14:50:00,2.0,2022-04-24 23:06:00,85.0
5885,48a43a20ff56ce42,af4eb3a7c4c82877,60f7af654740a548,af4eb3a7c4c82877,Discontinued,Rate Change,2021-10-26 14:50:00,2.0,2022-04-24 23:06:00,85.0
5886,a7b309ece2a59015,af4eb3a7c4c82877,60f7af654740a548,af4eb3a7c4c82877,Discontinued,Rate Change,2021-10-26 14:50:00,2.0,2022-04-24 23:06:00,85.0


In [67]:
len(df_combined[df_combined[['MED_ACTION_TIME', 'RECORDED_TIME']].isna().any(axis=1)])

0

In [68]:
len(df_combined.isna().any(axis=1))

680

In [71]:
test = df_combined.drop('UNITS', axis=1)

In [72]:
len(test.isna().any(axis=1))

680

In [73]:
test

,ENC_TYPE_C,ENC_TYPE_NM,LOG_ID_x,MRN_x,ORDERING_DATE,ORDER_CLASS_NM,MEDICATION_ID,DISPLAY_NAME,MEDICATION_NM,START_DATE,...,ADMIN_SIG,DOSE_UNIT_NM,MED_ROUTE_NM,LOG_ID_y,MRN_y,FLO_NAME,FLO_DISPLAY_NAME,RECORD_TYPE_y,RECORDED_TIME,MEAS_VALUE
0,3,Hospital Encounter,f522ec2b491b3891,508c48705f768d30,2022-08-01 00:00:00,Inpatient,507985,norepinephrine (LEVOPHED) 4 mg in 250 mL NS (L...,NOREPINEPHRINE 4 MG/250 ML NS INFUSION (UCI),2022-08-01 00:00:00,...,2.0,mcg/min,IntraVENOUS,b999f36713878caa,508c48705f768d30,Data,MAP (mmHg),POST-OP,2022-09-16 21:47:00,77.0
1,3,Hospital Encounter,f522ec2b491b3891,508c48705f768d30,2022-08-01 00:00:00,Inpatient,507985,norepinephrine (LEVOPHED) 4 mg in 250 mL NS (L...,NOREPINEPHRINE 4 MG/250 ML NS INFUSION (UCI),2022-08-01 00:00:00,...,2.0,mcg/min,IntraVENOUS,b999f36713878caa,508c48705f768d30,Data,MAP (mmHg),POST-OP,2022-09-16 21:47:00,77.0
2,3,Hospital Encounter,f522ec2b491b3891,508c48705f768d30,2022-08-01 00:00:00,Inpatient,507985,norepinephrine (LEVOPHED) 4 mg in 250 mL NS (L...,NOREPINEPHRINE 4 MG/250 ML NS INFUSION (UCI),2022-08-01 00:00:00,...,2.0,mcg/min,IntraVENOUS,b999f36713878caa,508c48705f768d30,Data,MAP (mmHg),POST-OP,2022-09-16 21:47:00,77.0
3,3,Hospital Encounter,f522ec2b491b3891,508c48705f768d30,2022-08-01 00:00:00,Inpatient,507985,norepinephrine (LEVOPHED) 4 mg in 250 mL NS (L...,NOREPINEPHRINE 4 MG/250 ML NS INFUSION (UCI),2022-08-01 00:00:00,...,2.0,mcg/min,IntraVENOUS,b999f36713878caa,508c48705f768d30,Data,MAP (mmHg),POST-OP,2022-09-16 21:47:00,77.0
4,3,Hospital Encounter,f522ec2b491b3891,508c48705f768d30,2022-08-01 00:00:00,Inpatient,507985,norepinephrine (LEVOPHED) 4 mg in 250 mL NS (L...,NOREPINEPHRINE 4 MG/250 ML NS INFUSION (UCI),2022-08-01 00:00:00,...,4.0,mcg/min,IntraVENOUS,b999f36713878caa,508c48705f768d30,Data,MAP (mmHg),POST-OP,2022-09-16 21:47:00,77.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
675,3,Hospital Encounter,b999f36713878caa,508c48705f768d30,2022-09-02 00:00:00,Inpatient,507985,norepinephrine (LEVOPHED) 4 mg in 250 mL NS (L...,NOREPINEPHRINE 4 MG/250 ML NS INFUSION (UCI),2022-09-02 00:00:00,...,0.0,mcg/min,IntraVENOUS,b999f36713878caa,508c48705f768d30,Data,MAP (mmHg),POST-OP,2022-09-16 21:47:00,77.0
676,3,Hospital Encounter,a6782852ccd4551d,508c48705f768d30,2022-09-02 00:00:00,Inpatient,507985,norepinephrine (LEVOPHED) 4 mg in 250 mL NS (L...,NOREPINEPHRINE 4 MG/250 ML NS INFUSION (UCI),2022-09-02 00:00:00,...,0.0,mcg/min,IntraVENOUS,b999f36713878caa,508c48705f768d30,Data,MAP (mmHg),POST-OP,2022-09-16 21:47:00,77.0
677,3,Hospital Encounter,2bc05f5dd6262613,508c48705f768d30,2022-09-02 00:00:00,Inpatient,507985,norepinephrine (LEVOPHED) 4 mg in 250 mL NS (L...,NOREPINEPHRINE 4 MG/250 ML NS INFUSION (UCI),2022-09-02 00:00:00,...,0.0,mcg/min,IntraVENOUS,b999f36713878caa,508c48705f768d30,Data,MAP (mmHg),POST-OP,2022-09-16 21:47:00,77.0
678,3,Hospital Encounter,eca1be1182a10d1e,508c48705f768d30,2022-09-02 00:00:00,Inpatient,507985,norepinephrine (LEVOPHED) 4 mg in 250 mL NS (L...,NOREPINEPHRINE 4 MG/250 ML NS INFUSION (UCI),2022-09-02 00:00:00,...,0.0,mcg/min,IntraVENOUS,b999f36713878caa,508c48705f768d30,Data,MAP (mmHg),POST-OP,2022-09-16 21:47:00,77.0


In [66]:
df_combined[['MED_ACTION_TIME', 'RECORDED_TIME']].isna().any()

MED_ACTION_TIME    False
RECORDED_TIME      False
dtype: bool

In [61]:
df_combined.columns

Index(['ENC_TYPE_C', 'ENC_TYPE_NM', 'LOG_ID_x', 'MRN_x', 'ORDERING_DATE',
       'ORDER_CLASS_NM', 'MEDICATION_ID', 'DISPLAY_NAME', 'MEDICATION_NM',
       'START_DATE', 'END_DATE', 'ORDER_STATUS_NM', 'RECORD_TYPE_x',
       'MAR_ACTION_NM', 'MED_ACTION_TIME', 'ADMIN_SIG', 'DOSE_UNIT_NM',
       'MED_ROUTE_NM', 'LOG_ID_y', 'MRN_y', 'FLO_NAME', 'FLO_DISPLAY_NAME',
       'RECORD_TYPE_y', 'RECORDED_TIME', 'MEAS_VALUE', 'UNITS'],
      dtype='object')

In [56]:
df_combined

,ENC_TYPE_C,ENC_TYPE_NM,LOG_ID_x,MRN_x,ORDERING_DATE,ORDER_CLASS_NM,MEDICATION_ID,DISPLAY_NAME,MEDICATION_NM,START_DATE,...,DOSE_UNIT_NM,MED_ROUTE_NM,LOG_ID_y,MRN_y,FLO_NAME,FLO_DISPLAY_NAME,RECORD_TYPE_y,RECORDED_TIME,MEAS_VALUE,UNITS
0,3,Hospital Encounter,f522ec2b491b3891,508c48705f768d30,2022-08-01 00:00:00,Inpatient,507985,norepinephrine (LEVOPHED) 4 mg in 250 mL NS (L...,NOREPINEPHRINE 4 MG/250 ML NS INFUSION (UCI),2022-08-01 00:00:00,...,mcg/min,IntraVENOUS,b999f36713878caa,508c48705f768d30,Data,MAP (mmHg),POST-OP,2022-09-16 21:47:00,77.0,NaN
1,3,Hospital Encounter,f522ec2b491b3891,508c48705f768d30,2022-08-01 00:00:00,Inpatient,507985,norepinephrine (LEVOPHED) 4 mg in 250 mL NS (L...,NOREPINEPHRINE 4 MG/250 ML NS INFUSION (UCI),2022-08-01 00:00:00,...,mcg/min,IntraVENOUS,b999f36713878caa,508c48705f768d30,Data,MAP (mmHg),POST-OP,2022-09-16 21:47:00,77.0,NaN
2,3,Hospital Encounter,f522ec2b491b3891,508c48705f768d30,2022-08-01 00:00:00,Inpatient,507985,norepinephrine (LEVOPHED) 4 mg in 250 mL NS (L...,NOREPINEPHRINE 4 MG/250 ML NS INFUSION (UCI),2022-08-01 00:00:00,...,mcg/min,IntraVENOUS,b999f36713878caa,508c48705f768d30,Data,MAP (mmHg),POST-OP,2022-09-16 21:47:00,77.0,NaN
3,3,Hospital Encounter,f522ec2b491b3891,508c48705f768d30,2022-08-01 00:00:00,Inpatient,507985,norepinephrine (LEVOPHED) 4 mg in 250 mL NS (L...,NOREPINEPHRINE 4 MG/250 ML NS INFUSION (UCI),2022-08-01 00:00:00,...,mcg/min,IntraVENOUS,b999f36713878caa,508c48705f768d30,Data,MAP (mmHg),POST-OP,2022-09-16 21:47:00,77.0,NaN
4,3,Hospital Encounter,f522ec2b491b3891,508c48705f768d30,2022-08-01 00:00:00,Inpatient,507985,norepinephrine (LEVOPHED) 4 mg in 250 mL NS (L...,NOREPINEPHRINE 4 MG/250 ML NS INFUSION (UCI),2022-08-01 00:00:00,...,mcg/min,IntraVENOUS,b999f36713878caa,508c48705f768d30,Data,MAP (mmHg),POST-OP,2022-09-16 21:47:00,77.0,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
675,3,Hospital Encounter,b999f36713878caa,508c48705f768d30,2022-09-02 00:00:00,Inpatient,507985,norepinephrine (LEVOPHED) 4 mg in 250 mL NS (L...,NOREPINEPHRINE 4 MG/250 ML NS INFUSION (UCI),2022-09-02 00:00:00,...,mcg/min,IntraVENOUS,b999f36713878caa,508c48705f768d30,Data,MAP (mmHg),POST-OP,2022-09-16 21:47:00,77.0,NaN
676,3,Hospital Encounter,a6782852ccd4551d,508c48705f768d30,2022-09-02 00:00:00,Inpatient,507985,norepinephrine (LEVOPHED) 4 mg in 250 mL NS (L...,NOREPINEPHRINE 4 MG/250 ML NS INFUSION (UCI),2022-09-02 00:00:00,...,mcg/min,IntraVENOUS,b999f36713878caa,508c48705f768d30,Data,MAP (mmHg),POST-OP,2022-09-16 21:47:00,77.0,NaN
677,3,Hospital Encounter,2bc05f5dd6262613,508c48705f768d30,2022-09-02 00:00:00,Inpatient,507985,norepinephrine (LEVOPHED) 4 mg in 250 mL NS (L...,NOREPINEPHRINE 4 MG/250 ML NS INFUSION (UCI),2022-09-02 00:00:00,...,mcg/min,IntraVENOUS,b999f36713878caa,508c48705f768d30,Data,MAP (mmHg),POST-OP,2022-09-16 21:47:00,77.0,NaN
678,3,Hospital Encounter,eca1be1182a10d1e,508c48705f768d30,2022-09-02 00:00:00,Inpatient,507985,norepinephrine (LEVOPHED) 4 mg in 250 mL NS (L...,NOREPINEPHRINE 4 MG/250 ML NS INFUSION (UCI),2022-09-02 00:00:00,...,mcg/min,IntraVENOUS,b999f36713878caa,508c48705f768d30,Data,MAP (mmHg),POST-OP,2022-09-16 21:47:00,77.0,NaN


In [32]:
temp

,HOSP_ADMSN_TIME,HOSP_DISCH_TIME
36051,2021-09-15 12:52:00,2022-05-17 10:40:00
61007,2021-01-13 05:04:00,2021-01-22 19:45:00


In [28]:
row['HOSP_ADMSN_TIME']

Timestamp('2021-01-13 05:04:00')

In [29]:
row['HOSP_DISCH_TIME']

Timestamp('2021-01-22 19:45:00')

In [31]:
df_map_p

,LOG_ID,MRN,FLO_NAME,FLO_DISPLAY_NAME,RECORD_TYPE,RECORDED_TIME,MEAS_VALUE,UNITS
Unnamed: 0,,,,,,,,
2556,33163fa8e730e747,af4eb3a7c4c82877,Vital Signs,MAP (mmHg),POST-OP,2021-10-04 03:00:00,93.0,NaN
2557,48a43a20ff56ce42,af4eb3a7c4c82877,Vital Signs,MAP (mmHg),POST-OP,2021-10-04 03:00:00,93.0,NaN
2565,fe48717c3d9c8a15,af4eb3a7c4c82877,Vital Signs,MAP (mmHg),POST-OP,2021-10-04 03:00:00,93.0,NaN
3282,48a43a20ff56ce42,af4eb3a7c4c82877,Vital Signs,MAP (mmHg),POST-OP,2021-10-04 04:00:00,98.0,NaN
3283,fe48717c3d9c8a15,af4eb3a7c4c82877,Vital Signs,MAP (mmHg),POST-OP,2021-10-04 04:00:00,98.0,NaN
...,...,...,...,...,...,...,...,...
56540330,a7b309ece2a59015,af4eb3a7c4c82877,CCP Vital Signs,MAP (mmHg),POST-OP,2022-02-19 11:50:00,88.0,NaN
56540331,48a43a20ff56ce42,af4eb3a7c4c82877,CCP Vital Signs,MAP (mmHg),POST-OP,2022-02-19 11:50:00,88.0,NaN
56540332,fe48717c3d9c8a15,af4eb3a7c4c82877,CCP Vital Signs,MAP (mmHg),POST-OP,2022-02-19 11:50:00,88.0,NaN


In [27]:
df_meds_p

,ENC_TYPE_C,ENC_TYPE_NM,LOG_ID,MRN,ORDERING_DATE,ORDER_CLASS_NM,MEDICATION_ID,DISPLAY_NAME,MEDICATION_NM,START_DATE,END_DATE,ORDER_STATUS_NM,RECORD_TYPE,MAR_ACTION_NM,MED_ACTION_TIME,ADMIN_SIG,DOSE_UNIT_NM,MED_ROUTE_NM
16577034,3,Hospital Encounter,33163fa8e730e747,af4eb3a7c4c82877,2021-10-25 00:00:00,Inpatient,507985,norepinephrine (LEVOPHED) 4 mg in 250 mL NS (L...,NOREPINEPHRINE 4 MG/250 ML NS INFUSION (UCI),2021-10-25 00:00:00,2021-10-28 00:00:00,Discontinued,POST-OP,Rate Change,2021-10-25 10:00:00,5.0,mcg/min,IntraVENOUS
16577035,3,Hospital Encounter,33163fa8e730e747,af4eb3a7c4c82877,2021-10-25 00:00:00,Inpatient,507985,norepinephrine (LEVOPHED) 4 mg in 250 mL NS (L...,NOREPINEPHRINE 4 MG/250 ML NS INFUSION (UCI),2021-10-25 00:00:00,2021-10-28 00:00:00,Discontinued,POST-OP,Rate Change,2021-10-25 16:00:00,4.0,mcg/min,IntraVENOUS
16577036,3,Hospital Encounter,33163fa8e730e747,af4eb3a7c4c82877,2021-10-25 00:00:00,Inpatient,507985,norepinephrine (LEVOPHED) 4 mg in 250 mL NS (L...,NOREPINEPHRINE 4 MG/250 ML NS INFUSION (UCI),2021-10-25 00:00:00,2021-10-28 00:00:00,Discontinued,POST-OP,Rate Change,2021-10-25 18:30:00,1.0,mcg/min,IntraVENOUS
16577037,3,Hospital Encounter,33163fa8e730e747,af4eb3a7c4c82877,2021-10-25 00:00:00,NaN,507885,norepinephrine (LEVOPHED) 4 mg in 250 mL NS 0....,norepinephrine (LEVOPHED) 4 mg in 250 mL NS 0....,2021-10-25 00:00:00,2021-10-26 00:00:00,Discontinued,POST-OP,Rate Change,2021-10-25 10:00:00,5.0,mcg/min,IntraVENOUS
16580351,3,Hospital Encounter,33163fa8e730e747,af4eb3a7c4c82877,2021-10-12 00:00:00,Inpatient,507985,norepinephrine (LEVOPHED) 4 mg in 250 mL NS (L...,NOREPINEPHRINE 4 MG/250 ML NS INFUSION (UCI),2021-10-12 00:00:00,2021-10-12 00:00:00,Discontinued,POST-OP,Stopped,2021-10-12 12:28:00,0.0,mcg/min,IntraVENOUS
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
23684657,3,Hospital Encounter,60f7af654740a548,af4eb3a7c4c82877,2021-10-25 00:00:00,Inpatient,507985,norepinephrine (LEVOPHED) 4 mg in 250 mL NS (L...,NOREPINEPHRINE 4 MG/250 ML NS INFUSION (UCI),2021-10-25 00:00:00,2021-10-28 00:00:00,Discontinued,POST-OP,Rate Change,2021-10-25 16:00:00,4.0,mcg/min,IntraVENOUS
23684658,3,Hospital Encounter,60f7af654740a548,af4eb3a7c4c82877,2021-10-25 00:00:00,NaN,507885,norepinephrine (LEVOPHED) 4 mg in 250 mL NS 0....,norepinephrine (LEVOPHED) 4 mg in 250 mL NS 0....,2021-10-25 00:00:00,2021-10-26 00:00:00,Discontinued,POST-OP,Rate Change,2021-10-25 10:00:00,5.0,mcg/min,IntraVENOUS
23700494,3,Hospital Encounter,a7b309ece2a59015,af4eb3a7c4c82877,2021-10-25 00:00:00,Inpatient,507985,norepinephrine (LEVOPHED) 4 mg in 250 mL NS (L...,NOREPINEPHRINE 4 MG/250 ML NS INFUSION (UCI),2021-10-25 00:00:00,2021-10-28 00:00:00,Discontinued,POST-OP,Restarted,2021-10-26 14:30:00,4.0,mcg/min,IntraVENOUS
23707433,3,Hospital Encounter,60f7af654740a548,af4eb3a7c4c82877,2021-10-25 00:00:00,Inpatient,507985,norepinephrine (LEVOPHED) 4 mg in 250 mL NS (L...,NOREPINEPHRINE 4 MG/250 ML NS INFUSION (UCI),2021-10-25 00:00:00,2021-10-28 00:00:00,Discontinued,POST-OP,Rate Change,2021-10-25 18:30:00,1.0,mcg/min,IntraVENOUS


In [35]:
df_hours = pd.read_csv('hours_df.csv', index_col=0)

In [37]:
df_hours['MRN'].nunique()

254

In [30]:
df_hours_new = pd.read_csv('hours_fb_df_new.csv', index_col=0)

In [33]:
df_hours_new['MRN'].nunique()

2171

In [31]:
df_hours_new

,MRN,HOSP_ADMSN_TIME,HOSP_DISCH_TIME,HOURS
0,38f2abdfacbed586,2021-08-24 07:19:00,2021-09-06 14:56:00,0.000000
1,2d604e5cc7818233,2022-08-08 22:20:00,2022-08-23 15:28:00,0.000000
2,3b7fd5bd48dae180,2022-01-20 01:47:00,2022-01-31 15:39:00,0.000000
3,0ea0ab0b01b01c4c,2019-01-12 18:19:00,2019-02-07 18:14:00,0.000000
4,bba97a9988e7a2aa,2019-03-24 04:07:00,2019-06-22 18:50:00,0.000000
...,...,...,...,...
2209,bb2936abd9e070d7,2021-10-21 15:06:00,2021-10-26 00:59:00,0.037777
2210,224c476538275696,2022-06-04 22:17:00,2022-07-17 21:59:00,0.000000
2211,5800610092b18862,2019-09-22 07:41:00,2019-11-24 17:05:00,0.540456
2212,5800610092b18862,2020-01-31 19:26:00,2020-03-10 11:25:00,0.000000


In [32]:
df_hours_new[df_hours_new['HOURS'] != 0]

,MRN,HOSP_ADMSN_TIME,HOSP_DISCH_TIME,HOURS
19,e51b5338f90df1a1,2020-02-29 18:04:00,2020-03-18 14:38:00,0.532550
26,e0f4cae6b5e41fef,2022-08-06 21:30:00,2022-09-24 13:34:00,0.016052
34,44ab0ccdab253f54,2019-02-10 23:33:00,2019-03-22 20:30:00,0.556717
36,0643e16f5e17692d,2022-05-30 04:48:00,2022-08-29 15:40:00,0.194089
37,f5f953458157ac36,2020-06-26 05:49:00,2020-07-26 19:02:00,0.989817
...,...,...,...,...
2199,b231c2078e38435b,2021-08-10 11:26:00,2021-08-22 12:15:00,0.083098
2200,b231c2078e38435b,2021-03-21 11:53:00,2021-04-13 22:40:00,0.165250
2206,0dda25cff5a027b0,2023-06-02 22:24:00,2023-06-22 15:10:00,0.540965
2209,bb2936abd9e070d7,2021-10-21 15:06:00,2021-10-26 00:59:00,0.037777


In [38]:
df_hours

,MRN,HOSP_ADMSN_TIME,HOSP_DISCH_TIME,HOURS
0,78825e2daf0745c7,2021-03-19 16:06:00,2021-04-22 20:10:00,0.055991
1,dcc0236b5d1a2d78,2019-12-30 09:29:00,2020-01-07 11:50:00,0.000000
2,a2a0faad3cba2169,2021-12-04 14:45:00,2022-01-28 20:22:00,0.078265
3,56a8bdc5d5a722ab,2019-04-08 06:19:00,2019-04-13 18:06:00,0.060706
4,395611a1441fc16e,2020-11-07 13:20:00,2020-12-08 16:30:00,0.012202
...,...,...,...,...
262,1705793fee3a0da1,2022-06-21 14:55:00,2022-07-07 20:25:00,0.513479
263,0a54f7a2cd0fac6f,2021-03-14 13:14:00,2021-03-19 17:15:00,0.111544
264,1443fbf786017a0a,2019-08-14 17:38:00,2019-09-02 17:30:00,0.012065
265,48fd4b2e04eca5f1,2022-03-29 10:48:00,2022-04-28 12:07:00,0.930266


In [203]:
import warnings
warnings.filterwarnings('ignore')

df_final_hours = pd.DataFrame(columns=['MRN','HOSP_ADMSN_TIME', 'HOSP_DISCH_TIME', 'HOURS'])
df_collection = []
count = 0
thresh = 10
for mrn in mrns:
    print(mrn)
    temp = df_patient[df_patient['MRN'] == mrn]
    temp = temp[['HOSP_ADMSN_TIME', 'HOSP_DISCH_TIME']].drop_duplicates()
    temp['HOSP_ADMSN_TIME'] = pd.to_datetime(temp['HOSP_ADMSN_TIME'])
    temp['HOSP_DISCH_TIME'] = pd.to_datetime(temp['HOSP_DISCH_TIME'])
    print(len(temp))
    for index, row in temp.iterrows():
        pmeds = df_meds[(df_meds['MRN'] == mrn) & (df_meds['MED_ACTION_TIME'] > row['HOSP_ADMSN_TIME']) 
                                                & (df_meds['MED_ACTION_TIME'] < row['HOSP_DISCH_TIME'])]
        pmaps = df_map[(df_map['MRN'] == mrn) & (df_map['RECORDED_TIME'] > row['HOSP_ADMSN_TIME']) 
                                                & (df_map['RECORDED_TIME'] < row['HOSP_DISCH_TIME'])]
        
        pmeds = pmeds.sort_values('MED_ACTION_TIME').reset_index(drop=True)
        pmaps = pmaps.sort_values('RECORDED_TIME').reset_index(drop=True)
        if pmeds.empty or pmaps.empty:
            print("No valid map or med meausrements")
            continue
        
        time_to_use = str()
        if len(pmeds) > len(pmaps):
            time_to_use = 'MED_ACTION_TIME'
            df_combined = pd.merge_asof(pmeds, pmaps, left_on='MED_ACTION_TIME', right_on='RECORDED_TIME', direction='backward')
        else:
            time_to_use = 'RECORDED_TIME'
            df_combined = pd.merge_asof(pmaps, pmeds, left_on='RECORDED_TIME', right_on='MED_ACTION_TIME', direction='backward')
        df_combined_cleaned = df_combined.dropna(subset=['MED_ACTION_TIME', 'RECORDED_TIME'])
        df_combined_cleaned['pressor_diff'] = df_combined_cleaned.groupby('MEDICATION_NM')['ADMIN_SIG'].diff()
        
        valid_timestamps = []
        for timestamp in df_combined_cleaned[time_to_use].unique():
            subset = df_combined_cleaned[df_combined_cleaned[time_to_use] == timestamp]
            if subset['MEAS_VALUE'].iloc[0] > 70:
                if (subset['pressor_diff'] < 0).any() and not (subset['pressor_diff'] > 0).any():
                    valid_timestamps.append(timestamp)
        df_valid = df_combined_cleaned[df_combined_cleaned[time_to_use].isin(valid_timestamps)]
        df_valid['time_diff'] = df_valid[time_to_use].diff().dt.total_seconds() / 3600
        total_hours = df_valid['time_diff'].sum()
        total_hours_hosp = (row['HOSP_DISCH_TIME'] - row['HOSP_ADMSN_TIME']).total_seconds() / 3600
        total_hours /= total_hours_hosp
        hours_dict = {
            'MRN': mrn,
            'HOSP_ADMSN_TIME': row['HOSP_ADMSN_TIME'],
            'HOSP_DISCH_TIME': row['HOSP_DISCH_TIME'],
            'HOURS': total_hours,
        }
        df_collection.append(hours_dict)
        print(f'total hours: {total_hours}')
        print(f"total hours in hospital: {total_hours_hosp}")
        
        #df_combined_cleaned['valid'] = (df_combined_cleaned['MEAS_VALUE'] > 70) & (df_combined_cleaned['pressor_diff'].fillna(0) == 0)
        # number 
#         map_over_thresh = df_combined_cleaned[df_combined_cleaned['MEAS_VALUE'] >= 70]
#         pressors = df_combined_cleaned['MEDICATION_NM'].unique()
#         lst = []
#         for pressor in pressors:
#             print(pressor)
#             df_filtered = df_combined_cleaned[df_combined_cleaned['MEDICATION_NM'] == pressor]
#             df_filtered[f'{pressor}_diff'] = df_combined_cleaned['ADMIN_SIG'].diff()
#             lst.append(np.array(df_filtered[f'{pressor}_diff']))
        count += 1
        if count >= thresh:
            break
    if count >= thresh:
        break

78825e2daf0745c7
1
total hours: 0.05599138281440533
total hours in hospital: 820.0666666666667
dcc0236b5d1a2d78
2
total hours: 0.0
total hours in hospital: 194.35
No valid map or med meausrements
29820d94cb18d909
2
No valid map or med meausrements
No valid map or med meausrements
a2a0faad3cba2169
1
total hours: 0.07826546135760716
total hours in hospital: 1325.6166666666666
56a8bdc5d5a722ab
1
total hours: 0.06070570380675351
total hours in hospital: 131.78333333333333
395611a1441fc16e
1
total hours: 0.012201650680347982
total hours in hospital: 747.1666666666666
642f27f1c2762314
4
No valid map or med meausrements
No valid map or med meausrements
No valid map or med meausrements
total hours: 0.0
total hours in hospital: 100.43333333333334
0be606e136c802d7
1
total hours: 0.0
total hours in hospital: 272.85
42329c8e42baa9a7
1
total hours: 0.0
total hours in hospital: 536.1833333333333
b653f7623ac3de28
1
total hours: 0.10547477942717762
total hours in hospital: 874.6166666666667
3036573015

In [204]:
df = pd.DataFrame(df_collection)

In [176]:
pmaps

,LOG_ID,MRN,FLO_NAME,FLO_DISPLAY_NAME,RECORD_TYPE,RECORDED_TIME,MEAS_VALUE,UNITS


In [177]:
pmeds

,ENC_TYPE_C,ENC_TYPE_NM,LOG_ID,MRN,ORDERING_DATE,ORDER_CLASS_NM,MEDICATION_ID,DISPLAY_NAME,MEDICATION_NM,START_DATE,END_DATE,ORDER_STATUS_NM,RECORD_TYPE,MAR_ACTION_NM,MED_ACTION_TIME,ADMIN_SIG,DOSE_UNIT_NM,MED_ROUTE_NM
0,3,Hospital Encounter,fec0116b664f6eb8,29820d94cb18d909,2018-12-12 00:00:00,Inpatient,505700,DOPamine (INTROPIN) 800 mg/250 mL D5W PREMIX i...,DOPAMINE 800 MG/250 ML D5W PREMIX INFUSION (UCI),2018-12-12 00:00:00,2018-12-12 00:00:00,Discontinued,POST-OP,Missed,2018-12-12 15:45:00,0.000,mcg/kg/min,IntraVENOUS
1,3,Hospital Encounter,fec0116b664f6eb8,29820d94cb18d909,2018-12-12 00:00:00,Inpatient,455306,DOBUTamine (DOBUTREX) 500 mg in sodium chlorid...,DOBUTAMINE 500 MG/250 ML NS INFUSION (UCI),2018-12-12 00:00:00,2018-12-18 00:00:00,Discontinued,POST-OP,New Bag,2018-12-12 16:00:00,2.000,mcg/kg/min,IntraVENOUS
2,3,Hospital Encounter,fec0116b664f6eb8,29820d94cb18d909,2018-12-12 00:00:00,Inpatient,455621,norepinephrine (LEVOPHED) 8 mg in dextrose 5 %...,NOREPINEPHRINE 8 MG/250 ML D5W INFUSION (NON-S...,2018-12-12 00:00:00,2018-12-16 00:00:00,Discontinued,POST-OP,New Bag,2018-12-12 16:45:00,2.000,mcg/min,IntraVENOUS
3,3,Hospital Encounter,fec0116b664f6eb8,29820d94cb18d909,2018-12-12 00:00:00,Inpatient,455306,DOBUTamine (DOBUTREX) 500 mg in sodium chlorid...,DOBUTAMINE 500 MG/250 ML NS INFUSION (UCI),2018-12-12 00:00:00,2018-12-18 00:00:00,Discontinued,POST-OP,New Bag,2018-12-12 17:00:00,2.000,mcg/kg/min,IntraVENOUS
4,3,Hospital Encounter,fec0116b664f6eb8,29820d94cb18d909,2018-12-12 00:00:00,Inpatient,455621,norepinephrine (LEVOPHED) 8 mg in dextrose 5 %...,NOREPINEPHRINE 8 MG/250 ML D5W INFUSION (NON-S...,2018-12-12 00:00:00,2018-12-16 00:00:00,Discontinued,POST-OP,Stopped,2018-12-12 17:00:00,0.000,mcg/min,IntraVENOUS
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
82,52,Anesthesia,fec0116b664f6eb8,29820d94cb18d909,2018-12-16 00:00:00,Inpatient,455651,DOBUTamine (DOBUTREX) 500 mg/250 mL infusion (...,DOBUTAMINE 500 MG/250 ML D5W PREMIX INFUSION (...,2018-12-16 00:00:00,2018-12-16 00:00:00,Discontinued,POST-OP,Rate Change,2018-12-16 08:58:00,2.000,mcg/kg/min,IntraVENOUS
83,52,Anesthesia,fec0116b664f6eb8,29820d94cb18d909,2018-12-16 00:00:00,Inpatient,455651,DOBUTamine (DOBUTREX) 500 mg/250 mL infusion (...,DOBUTAMINE 500 MG/250 ML D5W PREMIX INFUSION (...,2018-12-16 00:00:00,2018-12-16 00:00:00,Discontinued,POST-OP,Rate Change,2018-12-16 09:03:00,3.000,mcg/kg/min,IntraVENOUS
84,52,Anesthesia,fec0116b664f6eb8,29820d94cb18d909,2018-12-16 00:00:00,Inpatient,452941,EPINEPHrine 4 mg in sodium chloride 0.9 % 250 ...,EPINEPHRINE INFUSION (ANESTHESIA),2018-12-16 00:00:00,2018-12-16 00:00:00,Discontinued,POST-OP,Restarted,2018-12-16 09:15:00,1.000,mcg/kg/min,IntraVENOUS
85,52,Anesthesia,fec0116b664f6eb8,29820d94cb18d909,2018-12-16 00:00:00,Inpatient,452941,EPINEPHrine 4 mg in sodium chloride 0.9 % 250 ...,EPINEPHRINE INFUSION (ANESTHESIA),2018-12-16 00:00:00,2018-12-16 00:00:00,Discontinued,POST-OP,Stopped,2018-12-16 10:20:00,0.000,mcg/kg/min,IntraVENOUS


In [164]:
row['HOSP_ADMSN_TIME']

Timestamp('2018-12-08 17:30:00')

In [165]:
row['HOSP_DISCH_TIME']

Timestamp('2018-12-22 12:56:00')

In [167]:
total_hours

0.0

In [168]:
look = df_valid[['RECORDED_TIME', 'MED_ACTION_TIME', 'MEDICATION_NM', 'ADMIN_SIG', 'MEAS_VALUE', 'pressor_diff', 'pressor_diff']]

In [169]:
look

,RECORDED_TIME,MED_ACTION_TIME,MEDICATION_NM,ADMIN_SIG,MEAS_VALUE,pressor_diff,pressor_diff


In [174]:
df_combined_cleaned

,ENC_TYPE_C,ENC_TYPE_NM,LOG_ID_x,MRN_x,ORDERING_DATE,ORDER_CLASS_NM,MEDICATION_ID,DISPLAY_NAME,MEDICATION_NM,START_DATE,...,MED_ROUTE_NM,LOG_ID_y,MRN_y,FLO_NAME,FLO_DISPLAY_NAME,RECORD_TYPE_y,RECORDED_TIME,MEAS_VALUE,UNITS,pressor_diff


In [170]:
df_combined_cleaned['ADMIN_SIG'].value_counts()

Series([], Name: count, dtype: int64)

In [173]:
df_combined_cleaned[['RECORDED_TIME', 'MED_ACTION_TIME', 'MEDICATION_NM', 'ADMIN_SIG', 'MEAS_VALUE', 'pressor_diff', 'pressor_diff']]

,RECORDED_TIME,MED_ACTION_TIME,MEDICATION_NM,ADMIN_SIG,MEAS_VALUE,pressor_diff,pressor_diff


In [172]:
df_combined_cleaned['MEDICATION_NM'].value_counts()

Series([], Name: count, dtype: int64)

In [74]:
lst

[array([nan,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,
         0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.]),
 array([ 4.,  0.,  0.,  0.,  0.,  0., -3., -1.,  0.,  0.,  0.,  0.,  0.,
         0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,
         0.,  0.,  2., -2.,  2.,  0.,  0.,  1.,  0.,  0.,  0., -1.,  0.,
        -1.,  0.,  0., -1.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,
         0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,
         0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,
         0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,
         0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,
         0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,
         0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,
         0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,
         0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.,  0.])]

In [72]:
df_filtered

,LOG_ID_x,MRN_x,FLO_NAME,FLO_DISPLAY_NAME,RECORD_TYPE_x,RECORDED_TIME,MEAS_VALUE,UNITS,ENC_TYPE_C,ENC_TYPE_NM,...,START_DATE,END_DATE,ORDER_STATUS_NM,RECORD_TYPE_y,MAR_ACTION_NM,MED_ACTION_TIME,ADMIN_SIG,DOSE_UNIT_NM,MED_ROUTE_NM,NOREPINEPHRINE 4 MG/250 ML NS INFUSION (UCI)_diff
35,5969cc053be0cad8,78825e2daf0745c7,Vital Signs,MAP (mmHg),POST-OP,2021-03-22 00:00:00,71.0,NaN,3.0,Hospital Encounter,...,2021-03-21 00:00:00,2021-03-23 00:00:00,Discontinued,POST-OP,Rate Verify,2021-03-22 00:00:00,4.0,mcg/min,IntraVENOUS,4.0
36,5969cc053be0cad8,78825e2daf0745c7,Vital Signs,MAP (mmHg),POST-OP,2021-03-22 06:00:00,76.0,NaN,3.0,Hospital Encounter,...,2021-03-21 00:00:00,2021-03-23 00:00:00,Discontinued,POST-OP,Rate Verify,2021-03-22 06:00:00,4.0,mcg/min,IntraVENOUS,0.0
37,5969cc053be0cad8,78825e2daf0745c7,Vital Signs,MAP (mmHg),POST-OP,2021-03-22 10:00:00,77.0,NaN,3.0,Hospital Encounter,...,2021-03-21 00:00:00,2021-03-23 00:00:00,Discontinued,POST-OP,Rate Change,2021-03-22 08:45:00,4.0,mcg/min,IntraVENOUS,0.0
38,5969cc053be0cad8,78825e2daf0745c7,Vital Signs,MAP (mmHg),POST-OP,2021-03-22 11:00:00,74.0,NaN,3.0,Hospital Encounter,...,2021-03-21 00:00:00,2021-03-23 00:00:00,Discontinued,POST-OP,Rate Change,2021-03-22 08:45:00,4.0,mcg/min,IntraVENOUS,0.0
39,5969cc053be0cad8,78825e2daf0745c7,Vital Signs,MAP (mmHg),POST-OP,2021-03-22 12:00:00,65.0,NaN,3.0,Hospital Encounter,...,2021-03-21 00:00:00,2021-03-23 00:00:00,Discontinued,POST-OP,Rate Change,2021-03-22 08:45:00,4.0,mcg/min,IntraVENOUS,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
183,4585ea00c7c83904,78825e2daf0745c7,CCP Vital Signs,MAP (mmHg),POST-OP,2021-04-21 18:20:00,97.0,NaN,3.0,Hospital Encounter,...,2021-03-23 00:00:00,2021-03-26 00:00:00,Discontinued,POST-OP,Rate Change,2021-03-24 13:25:00,0.0,mcg/min,IntraVENOUS,0.0
184,4585ea00c7c83904,78825e2daf0745c7,CCP Vital Signs,MAP (mmHg),POST-OP,2021-04-22 01:53:00,85.0,NaN,3.0,Hospital Encounter,...,2021-03-23 00:00:00,2021-03-26 00:00:00,Discontinued,POST-OP,Rate Change,2021-03-24 13:25:00,0.0,mcg/min,IntraVENOUS,0.0
185,4585ea00c7c83904,78825e2daf0745c7,CCP Vital Signs,MAP (mmHg),POST-OP,2021-04-22 06:30:00,87.0,NaN,3.0,Hospital Encounter,...,2021-03-23 00:00:00,2021-03-26 00:00:00,Discontinued,POST-OP,Rate Change,2021-03-24 13:25:00,0.0,mcg/min,IntraVENOUS,0.0
186,4585ea00c7c83904,78825e2daf0745c7,CCP Vital Signs,MAP (mmHg),POST-OP,2021-04-22 13:00:00,84.0,NaN,3.0,Hospital Encounter,...,2021-03-23 00:00:00,2021-03-26 00:00:00,Discontinued,POST-OP,Rate Change,2021-03-24 13:25:00,0.0,mcg/min,IntraVENOUS,0.0


In [66]:
df_combined_cleaned[['MED_ACTION_TIME', 'RECORDED_TIME']]

,MED_ACTION_TIME,RECORDED_TIME
13,2021-03-20 18:00:00,2021-03-21 00:00:00
14,2021-03-20 18:00:00,2021-03-21 01:00:00
15,2021-03-20 18:00:00,2021-03-21 02:00:00
16,2021-03-20 18:00:00,2021-03-21 03:00:00
17,2021-03-20 18:00:00,2021-03-21 04:00:00
...,...,...
183,2021-03-24 13:25:00,2021-04-21 18:20:00
184,2021-03-24 13:25:00,2021-04-22 01:53:00
185,2021-03-24 13:25:00,2021-04-22 06:30:00
186,2021-03-24 13:25:00,2021-04-22 13:00:00


In [59]:
list(pmaps['RECORDED_TIME'])

[Timestamp('2021-03-19 23:30:00'),
 Timestamp('2021-03-20 02:00:00'),
 Timestamp('2021-03-20 03:00:00'),
 Timestamp('2021-03-20 09:00:00'),
 Timestamp('2021-03-20 09:15:00'),
 Timestamp('2021-03-20 09:30:00'),
 Timestamp('2021-03-20 10:00:00'),
 Timestamp('2021-03-20 10:30:00'),
 Timestamp('2021-03-20 11:00:00'),
 Timestamp('2021-03-20 11:30:00'),
 Timestamp('2021-03-20 12:00:00'),
 Timestamp('2021-03-20 13:00:00'),
 Timestamp('2021-03-20 16:00:00'),
 Timestamp('2021-03-21 00:00:00'),
 Timestamp('2021-03-21 01:00:00'),
 Timestamp('2021-03-21 02:00:00'),
 Timestamp('2021-03-21 03:00:00'),
 Timestamp('2021-03-21 04:00:00'),
 Timestamp('2021-03-21 05:00:00'),
 Timestamp('2021-03-21 06:00:00'),
 Timestamp('2021-03-21 12:15:00'),
 Timestamp('2021-03-21 12:30:00'),
 Timestamp('2021-03-21 12:45:00'),
 Timestamp('2021-03-21 13:00:00'),
 Timestamp('2021-03-21 13:30:00'),
 Timestamp('2021-03-21 15:00:00'),
 Timestamp('2021-03-21 17:00:00'),
 Timestamp('2021-03-21 18:00:00'),
 Timestamp('2021-03-

In [21]:
df_merged['ADMIN_SIG'].value_counts()

ADMIN_SIG
0.0    156
3.0     31
Name: count, dtype: int64

In [104]:
df_patient

,LOG_ID,MRN,DISCH_DISP_C,DISCH_DISP,HOSP_ADMSN_TIME,HOSP_DISCH_TIME,LOS,ICU_ADMIN_FLAG,SURGERY_DATE,BIRTH_DATE,...,PRIMARY_ANES_TYPE_NM,ASA_RATING_C,ASA_RATING,PATIENT_CLASS_GROUP,PATIENT_CLASS_NM,PRIMARY_PROCEDURE_NM,IN_OR_DTTM,OUT_OR_DTTM,AN_START_DATETIME,AN_STOP_DATETIME
0,cd1636c6279d73a0,2e92522a5f2a2a22,15.0,Home Routine,12/20/18 11:20,12/21/18 12:15,1.0,No,12/20/18 0:00,47,...,General,3.0,Severe Systemic Disease,Outpatient,Hospital Outpatient Surgery,"INSERTION, PUBOVAGINAL SLING, WITH CYSTOSCOPY",12/20/18 12:27,12/20/18 17:25,12/20/18 12:27,12/20/18 17:34
1,fc53c06ee2cfe438,b7a91e623d957d8f,16.0,Hospice Facility,9/27/19 9:02,10/18/19 18:13,21.0,Yes,9/27/19 0:00,81,...,General,3.0,Severe Systemic Disease,Outpatient,Hospital Outpatient Surgery,GI EGD DILATION,9/27/19 9:52,9/27/19 11:27,9/27/19 9:52,9/27/19 11:44
2,2f0090c47da34608,b68d6c7198f30f73,15.0,Home Routine,4/8/19 9:33,4/11/19 12:34,3.0,Yes,4/8/19 0:00,49,...,General,2.0,Mild Systemic Disease,Inpatient,Hospital Inpatient Surgery,CRANIECTOMY,4/8/19 14:41,4/8/19 18:46,4/8/19 14:41,4/8/19 18:53
3,6d3b621880c6ced4,46aa48e89b9c50e8,15.0,Home Routine,12/3/18 5:57,12/4/18 12:57,1.0,No,12/3/18 0:00,54,...,General,2.0,Mild Systemic Disease,Outpatient,Hospital Outpatient Surgery,"REVISION, RECONSTRUCTION, BREAST",12/3/18 8:09,12/3/18 11:49,12/3/18 8:09,12/3/18 11:52
4,82f89e10f2855420,c2756216cc89b795,15.0,Home Routine,4/19/19 6:19,4/20/19 9:26,1.0,No,4/19/19 0:00,44,...,General,2.0,Mild Systemic Disease,Outpatient,Hospital Outpatient Surgery,"REVISION, RECONSTRUCTION, BREAST",4/19/19 8:13,4/19/19 14:51,4/19/19 8:14,4/19/19 15:00
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
65723,b619db184a3f7d86,aecf54e7e0077c0b,6.0,Skilled Nursing Facility,6/7/21 3:35,8/3/21 13:00,57.0,Yes,6/19/21 0:00,65,...,General,3.0,Severe Systemic Disease,Inpatient,Inpatient Admission,"EGD, WITH GASTROJEJUNOSTOMY TUBE INSERTION, PE...",6/19/21 7:12,6/19/21 8:22,6/19/21 7:12,6/19/21 8:27
65724,54b5138bdecfd66f,cd3f8576771bf9d2,15.0,Home Routine,11/11/21 13:14,11/11/21 20:14,0.0,No,11/11/21 0:00,48,...,General,2.0,Mild Systemic Disease,Outpatient,Hospital Outpatient Surgery,"HYSTEROSCOPY, WITH DILATION AND CURETTAGE OF U...",11/11/21 15:43,11/11/21 17:47,11/11/21 15:43,11/11/21 17:54
65725,35d4b39cdc9525ee,50b9fdfb13f2d8ec,15.0,Home Routine,3/7/22 11:24,3/8/22 11:30,1.0,Yes,3/7/22 0:00,62,...,General,3.0,Severe Systemic Disease,Outpatient,Hospital Outpatient Surgery,FOOT AND ANKLE - MASS OR FOREIGN BODY EXCISION,3/7/22 17:15,3/7/22 20:26,3/7/22 17:15,3/7/22 20:32
65726,399d457014a50d02,89d24047a41d7048,15.0,Home Routine,9/6/22 10:25,9/6/22 22:50,0.0,No,9/6/22 0:00,34,...,General,2.0,Mild Systemic Disease,Inpatient,Inpatient Admission,"CYSTOSCOPY, WITH URETERAL STENT INSERTION",9/6/22 20:54,9/6/22 21:49,9/6/22 20:54,9/6/22 21:53


In [105]:
df_patient[df_patient['MRN'] == '77bb0379da107da0']

,LOG_ID,MRN,DISCH_DISP_C,DISCH_DISP,HOSP_ADMSN_TIME,HOSP_DISCH_TIME,LOS,ICU_ADMIN_FLAG,SURGERY_DATE,BIRTH_DATE,...,PRIMARY_ANES_TYPE_NM,ASA_RATING_C,ASA_RATING,PATIENT_CLASS_GROUP,PATIENT_CLASS_NM,PRIMARY_PROCEDURE_NM,IN_OR_DTTM,OUT_OR_DTTM,AN_START_DATETIME,AN_STOP_DATETIME
2626,eee273c183272149,77bb0379da107da0,3.0,Expired,12/30/18 2:51,1/31/19 2:46,32.0,Yes,12/30/18 0:00,18,...,General,4.0,Incapacitating Disease,Inpatient,Inpatient Admission,IR ANGIO PELVIS,12/30/18 8:40,12/30/18 12:00,12/30/18 8:35,12/30/18 12:26
2672,a31e2721ec0a5974,77bb0379da107da0,3.0,Expired,12/30/18 2:51,1/31/19 2:46,32.0,Yes,1/3/19 0:00,18,...,General,4.0,Incapacitating Disease,Inpatient,Inpatient Admission,"INSERTION, INTRAMEDULLARY NAIL, TIBIA",1/3/19 17:13,1/4/19 0:00,1/3/19 17:14,1/4/19 0:24
2747,7ad9f0e2c9492683,77bb0379da107da0,3.0,Expired,12/30/18 2:51,1/31/19 2:46,32.0,Yes,1/7/19 0:00,18,...,General,5.0,Moribund,Inpatient,Inpatient Admission,"ORIF, FRACTURE, ACETABULUM",1/7/19 7:50,1/7/19 12:08,1/7/19 7:50,1/7/19 12:22
2957,1448853f211a1d17,77bb0379da107da0,3.0,Expired,12/30/18 2:51,1/31/19 2:46,32.0,Yes,1/10/19 0:00,18,...,General,4.0,Incapacitating Disease,Inpatient,Inpatient Admission,"DEBRIDEMENT, WOUND",1/10/19 7:20,1/10/19 11:49,1/10/19 7:20,1/10/19 12:02
3139,a644b0b810097e03,77bb0379da107da0,3.0,Expired,12/30/18 2:51,1/31/19 2:46,32.0,Yes,1/13/19 0:00,18,...,General,4.0,Incapacitating Disease,Inpatient,Inpatient Admission,"DEBRIDEMENT, WOUND",1/13/19 12:00,1/13/19 15:19,1/13/19 12:04,1/13/19 15:35
3297,ed66132faa7a2881,77bb0379da107da0,3.0,Expired,12/30/18 2:51,1/31/19 2:46,32.0,Yes,1/18/19 0:00,18,...,General,4.0,Incapacitating Disease,Inpatient,Inpatient Admission,"IRRIGATION AND DEBRIDEMENT, EXTREMITY",1/18/19 13:44,1/18/19 19:07,1/18/19 13:41,1/18/19 19:22
3441,961800ffe2704977,77bb0379da107da0,3.0,Expired,12/30/18 2:51,1/31/19 2:46,32.0,Yes,1/21/19 0:00,18,...,General,4.0,Incapacitating Disease,Inpatient,Inpatient Admission,"EGD, WITH GASTROJEJUNOSTOMY TUBE INSERTION, PE...",1/21/19 12:48,1/21/19 15:10,1/21/19 12:48,1/21/19 15:30
3506,9d563c76cae15442,77bb0379da107da0,3.0,Expired,12/30/18 2:51,1/31/19 2:46,32.0,Yes,1/24/19 0:00,18,...,General,4.0,Incapacitating Disease,Inpatient,Inpatient Admission,TRACHEOSTOMY,1/24/19 7:19,1/24/19 9:27,1/24/19 7:19,1/24/19 9:38
3605,8ff2e662881e897e,77bb0379da107da0,3.0,Expired,12/30/18 2:51,1/31/19 2:46,32.0,Yes,1/28/19 0:00,18,...,General,4.0,Incapacitating Disease,Inpatient,Inpatient Admission,"DEBRIDEMENT, WOUND",1/28/19 7:13,1/28/19 18:00,1/28/19 7:13,1/28/19 18:17
3687,4d364c7ab9846a85,77bb0379da107da0,3.0,Expired,12/30/18 2:51,1/31/19 2:46,32.0,Yes,1/26/19 0:00,18,...,General,4.0,Incapacitating Disease,Inpatient,Inpatient Admission,"ORIF, FRACTURE, ACETABULUM",1/26/19 7:13,1/26/19 11:32,1/26/19 7:18,1/26/19 11:49


In [ ]:
# Filter for patients with multiple separate procedures ()
# (1) map each LOG_ID to it's hospital start and end date